In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize']=(20,10)
%matplotlib inline

In [ ]:
df=pd.read_csv('../input/bengaluru-house-price-data/Bengaluru_House_Data.csv')
df.head()

In [ ]:
df.shape

delling with missing values

In [ ]:
df.isnull().sum()

In [ ]:
df1=df.dropna()
df1.isnull().sum()

dropping unnecessary features from dataframe

In [ ]:
df2=df1.drop(['availability','society','balcony','area_type'],axis='columns')
df2.head()

In [ ]:
df2['size'].unique()

create new feature for numbers of bedroom called bhk

In [ ]:
df3=df2.copy()
df3['bhk']=df2['size'].apply(lambda x: int(x.split(' ')[0]))
df3.head()

cleaning total_sqft

In [ ]:
df3.groupby('total_sqft')['total_sqft'].agg('count')

In [ ]:
def is_float(x):
    try:
        float(x)
    except:
        return False
    return True

In [ ]:
df3[~df3.total_sqft.apply(is_float)]

In [ ]:
def convert_sqft_to_no(x):
    token=x.split('-')
    if len(token)==2:
        return ((float(token[0])+float(token[1]))/2)
    try:
        return float(x)
    except:
        return None

In [ ]:
df4=df3.copy()
df4['total_sqft']=df3['total_sqft'].apply(convert_sqft_to_no)
df4.head()

add new feature (price_per_sqft)

In [ ]:
df5=df4.copy()
df5['price_per_sqft']=df5.price*100000/df5.total_sqft
df5.head()

strip any extra space in location column and remove locations have less than 10 data point

In [ ]:
df5.location=df5.location.apply(lambda x: x.strip())

In [ ]:
location_status=df5.groupby('location')['location'].agg('count').sort_values(ascending=False)
location_status

In [ ]:
location_less_than_10=location_status[location_status<=10]

In [ ]:
df6=df5.copy()
df6.location=df5.location.apply(lambda x: 'other' if x in location_less_than_10 else x)
df6.head()

detect outliers and remove them

In [ ]:
df7=df6[~(df6['total_sqft']/df6['bhk']<=300)]
df7.head()

In [ ]:
df7.price_per_sqft.describe()

In [ ]:
def remove_outliers(df):
    df_out=pd.DataFrame()
    for key,subdf in df.groupby('location'):
        m=np.mean(subdf.price_per_sqft)
        st=np.std(subdf.price_per_sqft)
        reduced_df=subdf[(subdf.price_per_sqft>(m-st))&(subdf.price_per_sqft<=(m+st))]
        df_out=pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

In [ ]:
df8=remove_outliers(df7)
df8.shape

In [ ]:
def plot_scatter_chart(df,location):
    bhk2=df[(df.location==location)&(df.bhk==2)]
    bhk3=df[(df.location==location)&(df.bhk==3)]
    matplotlib.rcParams['figure.figsize']=(15,10)
    plt.scatter(bhk2.total_sqft,bhk2.price,color='blue',label='2 BHK',s=50)
    plt.scatter(bhk3.total_sqft,bhk3.price,marker='+',color='green',label='3 BHK',s=50)
    plt.xlabel('Total Square area')
    plt.ylabel('Price per square feet')
    plt.title(location)
    plt.legend()

In [ ]:
plot_scatter_chart(df8,'Uttarahalli')

Notice that 2bhk price must be less than 3bhk price SO:

In [ ]:
def remove_bhk_outliers(df):
    exclude_indices=np.array([])
    for location,location_df in df.groupby('location'):
        bhk_stats={}
        for bhk,bhk_df in location_df.groupby('bhk'):
            bhk_stats[bhk]={
                'mean':np.mean(bhk_df.price_per_sqft),
                'std':np.std(bhk_df.price_per_sqft),
                'count':bhk_df.shape[0]
            }
        for bhk, bhk_df in location_df.groupby('bhk'):
            stats= bhk_stats.get(bhk-1)
            if stats and stats['count']>5:
                exclude_indices=np.append(exclude_indices, bhk_df[bhk_df.price_per_sqft<(stats['mean'])].index.values)
    return df.drop(exclude_indices, axis='index')

In [ ]:
df9=remove_bhk_outliers(df8)
df9.shape

after removing outliers

In [ ]:
plot_scatter_chart(df9,'Uttarahalli')

Notice that,numbers of bath must be less than bhk numbears SO:

In [ ]:
df9[df9['bath']>df9['bhk']+2].shape

In [ ]:
df10=df9[df9['bath']<df9['bhk']+2]
df10.head()

droping unnecessary features 

In [ ]:
df11=df10.drop(['size','price_per_sqft'],axis='columns')
df11.head(5)

prepare data for ml model

In [ ]:
dummies=pd.get_dummies(df11.location)
dummies.head()

In [ ]:
df12=pd.concat([df11.drop('location',axis='columns'),dummies.drop('other',axis='columns')],axis='columns')
df12.head()

In [ ]:
x=df12.drop('price',axis='columns')
y=df12.price

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=10)
from sklearn.linear_model import LinearRegression
lr_clf=LinearRegression()
lr_clf.fit(x_train,y_train)
lr_clf.score(x_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
cross_val_score(LinearRegression(),x,y,cv=cv)

know we want to find best regression  method 

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_linear_method(x,y):
    algos={
        'linear_regression':{
            'model': LinearRegression(),
            'params':{
                'normalize':[True,False]
            }
        },
        'lasso':{
            'model':Lasso(),
            'params':{
                'alpha':[1,2],
                'selection':['random','cyclic']
            }
        },
        'decision_tree':{
            'model':DecisionTreeRegressor(),
            'params':{
                'criterion':['mse','friedman_mse'],
                'splitter':['best','random']
            }
        }
    }
    scores=[]
    cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for algo_name,config in algos.items():
        gs=GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(x,y)
        scores.append({
            'model':algo_name,
            'best_score':gs.best_score_,
            'best_params':gs.best_params_
        })
    return pd.DataFrame(scores,columns=['model','best_score','best_params'])

In [ ]:
find_best_linear_method(x,y)

build prediction function for price

In [ ]:
def price_prediction(location,sqft,bath,bhk):
    loc_index=np.where(x.columns==location)[0][0]
    
    X=np.zeros(len(x.columns))
    X[0]=sqft
    X[1]=bath
    X[2]=bhk
    if loc_index>=0:
        X[loc_index]=1
    return lr_clf.predict([X])[0]

testing the function

In [ ]:
price_prediction('Thubarahalli',12000,5,3)